In [62]:
import torch
import numpy as np
import os
import pydicom as dicom
import pandas as pd
from xml.dom.minidom import parse


In [63]:
TRAIN_ON_GPU = torch.cuda.is_available()

if(TRAIN_ON_GPU):
    print('Training on GPU!')
else:
    print('Only CPU available')

Training on GPU!


In [64]:
seed = 41
batch_size = 20
image_size = 28

np.random.seed(seed)
torch.manual_seed(seed)

# Load Dataset

## Files

In [65]:
filepath_uid = "D:/Datasets/Lung-PET-CT-Dxr/Annotation"
metadatafile="D:/Datasets/Lung-PET-CT-Dxr/metadata.csv"
root="D:/Datasets/Lung-PET-CT-Dxr"

In [ ]:
# CSV-Datei als DataFrame laden
df = pd.read_csv(metadatafile)
# Wende die Formatierung auf die gesamte Spalte 'File Location' an
df['File Location'] = df['File Location'].apply(lambda x: x.lstrip('.').replace('//', '/'))
dfpet = df[df['Manufacturer']=="PT"]
dfct = df[df['Manufacturer']!="PT"]


In [67]:
# Initialisiere das Dictionary, um die Ergebnisse zu speichern
dicom_dict = {}

# Funktion, um die UID einer DICOM-Datei zu extrahieren
def get_dicom_uid(file_path):
    try:
        dicom_data = dicom.dcmread(file_path)
        return dicom_data.SOPInstanceUID  # Gibt die UID zurück
    except Exception as e:
        print(f"Fehler beim Lesen der DICOM-Datei {file_path}: {e}")
        return None


In [68]:
# Iteriere durch jede Zeile der 'File Location'-Spalte
for file in dfct['File Location']:
    # Erstelle den vollständigen Pfad
    full_path = root + file
   
    # Prüfe, ob der Ordner existiert
    if os.path.isdir(full_path):
        # print('Folder', full_path,'existiert' )
        # Liste alle DICOM-Dateien im Verzeichnis auf
        dicom_files = [f for f in os.listdir(full_path) if f.endswith('.dcm')]
        
        for dicom_file in dicom_files:
            dicom_file_path = os.path.join(full_path, dicom_file.replace("\\", "/"))
            
            # Holen der UID der DICOM-Datei
            dicom_uid = get_dicom_uid(dicom_file_path)
            
            if dicom_uid:
                # Speichern des Dateipfads und der UID im Dictionary
                dicom_dict[dicom_uid] = dicom_file_path.replace("\\", "/")
    else:
        print(f"Ordner {full_path} existiert nicht.")

# Ausgabe des resultierenden Dictionaries
# Umwandeln des Dictionaries in ein Pandas DataFrame
df_files = pd.DataFrame(list(dicom_dict.items()), columns=['UID', 'File Path'])




In [69]:
dicom_dict

{'1.3.6.1.4.1.14519.5.2.1.6655.2359.911257607240619696148712587012': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805/1-01.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.184801338381895107635381026857': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805/1-02.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.227252232966919521559290115017': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805/1-03.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.305763411933569638615761005863': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805/1-04.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.892649213333065045357131733969': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805/1-05.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.210616673074470338920284866510': 'D:/Datasets/Lun

## Annotations

In [70]:
# Initialisiere ein leeres Dictionary
annotation_dict = {}
# Iteriere durch alle Ordner und Dateien im Verzeichnis
for folder_name, subfolders, filenames in os.walk(filepath_uid):
    for filename in filenames:
        # Überprüfe, ob die Datei eine XML-Datei ist
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_name, filename)  # Erstelle den vollständigen Dateipfad
            
            # Der Dateiname ist die UID
            uid = os.path.splitext(filename)[0]  # Entferne die Erweiterung '.xml'
            
            # Füge UID und Dateipfad zum Dictionary hinzu
            annotation_dict[uid] = file_path.replace("\\", "/")

# Umwandeln des Dictionaries in ein Pandas DataFrame
df_annotations = pd.DataFrame(list(annotation_dict.items()), columns=['UID_Annotation', 'File Path_Annotation'])



In [71]:
# Listen für die Koordinaten
xmins, ymins, xmaxs, ymaxs = [], [], [], []

# Durch jede Zeile und jeden Dateipfad im DataFrame iterieren
for filepath in df_annotations['File Path_Annotation']:
    try:
        # XML-Datei parsen
        dom = parse(filepath)
        
        # Koordinaten extrahieren
        xmin = int(dom.getElementsByTagName('xmin')[0].firstChild.data)
        ymin = int(dom.getElementsByTagName('ymin')[0].firstChild.data)
        xmax = int(dom.getElementsByTagName('xmax')[0].firstChild.data)
        ymax = int(dom.getElementsByTagName('ymax')[0].firstChild.data)
        
        # Koordinaten zu den Listen hinzufügen
        xmins.append(xmin)
        ymins.append(ymin)
        xmaxs.append(xmax)
        ymaxs.append(ymax)
    except Exception as e:
        # Falls ein Fehler auftritt (z.B. Datei nicht gefunden), None hinzufügen
        xmins.append(None)
        ymins.append(None)
        xmaxs.append(None)
        ymaxs.append(None)
        print(f"Fehler bei Datei {filepath}: {e}")

# Neue Spalten für die Koordinaten im DataFrame hinzufügen
df_annotations['xmin'] = xmins
df_annotations['ymin'] = ymins
df_annotations['xmax'] = xmaxs
df_annotations['ymax'] = ymaxs

# Ergebnis anzeigen
df_annotations


Fehler bei Datei D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0024/1.3.6.1.4.1.14519.5.2.1.6655.2359.148117780944430339778694852523.xml: not well-formed (invalid token): line 1, column 0


,UID_Annotation,File Path_Annotation,xmin,ymin,xmax,ymax
0,1.3.6.1.4.1.14519.5.2.1.6655.2359.102500633407...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,286.0,310.0,355.0,402.0
1,1.3.6.1.4.1.14519.5.2.1.6655.2359.103293611003...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,304.0,304.0,337.0,372.0
2,1.3.6.1.4.1.14519.5.2.1.6655.2359.136943255924...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,278.0,308.0,360.0,394.0
3,1.3.6.1.4.1.14519.5.2.1.6655.2359.155870813347...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,290.0,305.0,338.0,378.0
4,1.3.6.1.4.1.14519.5.2.1.6655.2359.184131899543...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,301.0,329.0,351.0,372.0
...,...,...,...,...,...,...
31395,1.3.6.1.4.1.14519.5.2.1.6655.2359.922096117047...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,301.0,328.0,355.0,389.0
31396,1.3.6.1.4.1.14519.5.2.1.6655.2359.923461426394...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,296.0,278.0,400.0,380.0
31397,1.3.6.1.4.1.14519.5.2.1.6655.2359.968393286572...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,302.0,336.0,351.0,395.0
31398,1.3.6.1.4.1.14519.5.2.1.6655.2359.985887477459...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,306.0,321.0,386.0,408.0


## Delete Damaged Files

In [72]:
to_delet_ui = df_annotations['UID_Annotation'][df_annotations['File Path_Annotation'] == 'D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0024/1.3.6.1.4.1.14519.5.2.1.6655.2359.148117780944430339778694852523.xml']
to_delet_ui.iloc[0]


'1.3.6.1.4.1.14519.5.2.1.6655.2359.148117780944430339778694852523'

In [73]:
df_annotations = df_annotations[df_annotations['UID_Annotation'] != to_delet_ui.iloc[0]]
df_files=df_files[df_files['UID'] != to_delet_ui.iloc[0]]
df_annotations

,UID_Annotation,File Path_Annotation,xmin,ymin,xmax,ymax
0,1.3.6.1.4.1.14519.5.2.1.6655.2359.102500633407...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,286.0,310.0,355.0,402.0
1,1.3.6.1.4.1.14519.5.2.1.6655.2359.103293611003...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,304.0,304.0,337.0,372.0
2,1.3.6.1.4.1.14519.5.2.1.6655.2359.136943255924...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,278.0,308.0,360.0,394.0
3,1.3.6.1.4.1.14519.5.2.1.6655.2359.155870813347...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,290.0,305.0,338.0,378.0
4,1.3.6.1.4.1.14519.5.2.1.6655.2359.184131899543...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,301.0,329.0,351.0,372.0
...,...,...,...,...,...,...
31395,1.3.6.1.4.1.14519.5.2.1.6655.2359.922096117047...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,301.0,328.0,355.0,389.0
31396,1.3.6.1.4.1.14519.5.2.1.6655.2359.923461426394...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,296.0,278.0,400.0,380.0
31397,1.3.6.1.4.1.14519.5.2.1.6655.2359.968393286572...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,302.0,336.0,351.0,395.0
31398,1.3.6.1.4.1.14519.5.2.1.6655.2359.985887477459...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,306.0,321.0,386.0,408.0


## Save to CSV

In [74]:
# Speichern des DataFrames in einer CSV-Datei
csv_file_path = 'dicom_files.csv'
df_files.to_csv(csv_file_path, index=False)

print(f"Die File Daten wurden in die Datei {csv_file_path} geschrieben.")

# Speichern des DataFrames in einer CSV-Datei
csv_file_path = 'annotation_files.csv'
df_annotations.to_csv(csv_file_path, index=False)

print(f"Die Annotation Daten wurden in die Datei {csv_file_path} geschrieben.")


Die File Daten wurden in die Datei dicom_files.csv geschrieben.
Die Annotation Daten wurden in die Datei annotation_files.csv geschrieben.


In [77]:
df_annotations = pd.read_csv('C:/Users/chris/Documents/GitHub/Masterarbeit_Dominik/Christoph/Object_Detection/annotation_files.csv')
df_annotations

,UID_Annotation,File Path_Annotation,xmin,ymin,xmax,ymax
0,1.3.6.1.4.1.14519.5.2.1.6655.2359.102500633407...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,286.0,310.0,355.0,402.0
1,1.3.6.1.4.1.14519.5.2.1.6655.2359.103293611003...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,304.0,304.0,337.0,372.0
2,1.3.6.1.4.1.14519.5.2.1.6655.2359.136943255924...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,278.0,308.0,360.0,394.0
3,1.3.6.1.4.1.14519.5.2.1.6655.2359.155870813347...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,290.0,305.0,338.0,378.0
4,1.3.6.1.4.1.14519.5.2.1.6655.2359.184131899543...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1...,301.0,329.0,351.0,372.0
...,...,...,...,...,...,...
31394,1.3.6.1.4.1.14519.5.2.1.6655.2359.922096117047...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,301.0,328.0,355.0,389.0
31395,1.3.6.1.4.1.14519.5.2.1.6655.2359.923461426394...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,296.0,278.0,400.0,380.0
31396,1.3.6.1.4.1.14519.5.2.1.6655.2359.968393286572...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,302.0,336.0,351.0,395.0
31397,1.3.6.1.4.1.14519.5.2.1.6655.2359.985887477459...,D:/Datasets/Lung-PET-CT-Dxr/Annotation/G0062/1...,306.0,321.0,386.0,408.0


In [78]:
# CSV-Datei als DataFrame laden
df_uids = pd.read_csv('C:/Users/chris/Documents/GitHub/Masterarbeit_Dominik/Christoph/Object_Detection/dicom_files.csv')
df_uids

,UID,File Path
0,1.3.6.1.4.1.14519.5.2.1.6655.2359.911257607240...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
1,1.3.6.1.4.1.14519.5.2.1.6655.2359.184801338381...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
2,1.3.6.1.4.1.14519.5.2.1.6655.2359.227252232966...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
3,1.3.6.1.4.1.14519.5.2.1.6655.2359.305763411933...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
4,1.3.6.1.4.1.14519.5.2.1.6655.2359.892649213333...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
...,...,...
205454,1.3.6.1.4.1.14519.5.2.1.6655.2359.298663419902...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
205455,1.3.6.1.4.1.14519.5.2.1.6655.2359.296172154417...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
205456,1.3.6.1.4.1.14519.5.2.1.6655.2359.446085897895...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...
205457,1.3.6.1.4.1.14519.5.2.1.6655.2359.776270721996...,D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lun...


In [ ]:
df_merged = pd.merge(df_uids, df_annotations, left_on='UID', right_on='UID_Annotation', how='outer')
df_merged = df_merged.dropna(subset=['UID'])
# Neue Spalte 'Label' erstellen mit Bedingung
df_merged['Label'] = df_merged['UID_Annotation'].apply(lambda x: 'Negativ' if pd.isna(x) else 'Positiv')
df_merged.to_csv('merged.csv', index=False)


Label
Negativ    174721
Positiv     30738
Name: count, dtype: int64

# KP

In [49]:
image_directory = "D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001"
# annotation_directory = 'dataset/annot'
annotation_directory = "C:/Users/chris/Documents/GitHub/Master/Object_Localization/dataset/dataset/annot"

In [28]:
def loadFileInformation(filename):
    # Erstelle ein leeres Dictionary, um die extrahierten DICOM-Informationen zu speichern.
    information = {}
    
    # Lese die DICOM-Datei mit dem angegebenen Dateinamen ein.
    # dicom.dcmread lädt den Inhalt der DICOM-Datei in ein Dataset-Objekt 'ds'.
    # Das Argument `force=True` ermöglicht das Einlesen, auch wenn die Datei 
    # kleinere Inkonsistenzen oder Fehler im Format hat.
    ds = dicom.dcmread(filename, force=True)
    
    # Greife auf die SOPInstanceUID in den DICOM-Metadaten zu.
    # Die SOPInstanceUID ist eine eindeutige Kennung für diese Bildinstanz.
    # Speichere diese UID im Dictionary unter dem Schlüssel 'dicom_num'.
    information['dicom_num'] = ds.SOPInstanceUID
    
    # Gib das Dictionary zurück, das die UID (SOPInstanceUID) enthält.
    return information


In [18]:
def getUID_path(path):
    dict = {}  # Initialisiere ein leeres Wörterbuch, um UIDs und Dateipfade zu speichern.
    list = os.listdir(path)  # Liste alle Unterverzeichnisse im angegebenen 'path' auf.

    for date in list:  # Schleife über jedes Unterverzeichnis (z. B. ein Datum).
        date_path = os.path.join(path, date)  # Erstelle den vollständigen Pfad für das Datum.
        series_list = os.listdir(date_path)  # Liste alle Serien in diesem Datumspfad auf.
        series_list.sort()  # Sortiere die Serienliste, um eine feste Reihenfolge zu haben.

        for series in series_list:  # Schleife über jede Serie im aktuellen Datumspfad.
            series_path = os.path.join(date_path, series)  # Vollständiger Pfad für die Serie.
            dicom_list = os.listdir(series_path)  # Liste alle DICOM-Dateien in der Serie auf.
            dicom_list.sort()  # Sortiere die DICOM-Dateien, um eine feste Reihenfolge zu haben.

            for dicom in dicom_list:  # Schleife über jede DICOM-Datei in der Serie.
                dicom_path = os.path.join(series_path, dicom)  # Vollständiger Pfad für die DICOM-Datei.
                info = loadFileInformation(dicom_path)  # Lese DICOM-Informationen (einschließlich UID) aus.
                dict[info['dicom_num']] = (dicom_path, dicom)  # Speichere UID als Schlüssel mit Pfad und Dateiname.

    return dict  # Gib das Wörterbuch mit allen gefundenen UIDs und zugehörigen Dateipfaden zurück.


In [29]:
import os

def getUID_path(path):
    dict = {}  # Initialisiere ein leeres Wörterbuch, um UIDs und Dateipfade zu speichern.

    # Verwende os.walk, um alle Verzeichnisse und Dateien ab 'path' rekursiv zu durchsuchen.
    for root, dirs, files in os.walk(path):
        # root: aktuelles Verzeichnis
        # dirs: Liste der Unterverzeichnisse im aktuellen Verzeichnis
        # files: Liste der Dateien im aktuellen Verzeichnis

        # Sortiere die Verzeichnisse und Dateien alphabetisch (optional, für feste Reihenfolge).
        dirs.sort()
        files.sort()

        # Schleife durch jede Datei im aktuellen Verzeichnis
        for dicom in files:
            dicom_path = os.path.join(root, dicom)  # Vollständiger Pfad für die DICOM-Datei
            info = loadFileInformation(dicom_path)  # Lese DICOM-Informationen (einschließlich UID) aus
            dict[info['dicom_num']] = (dicom_path, dicom)  # Speichere UID als Schlüssel mit Pfad und Dateiname

    return dict  # Gib das Wörterbuch mit allen gefundenen UIDs und zugehörigen Dateipfaden zurück.


In [30]:
bilder=getUID_path(image_directory)

In [31]:
bilder

{'1.3.6.1.4.1.14519.5.2.1.6655.2359.911257607240619696148712587012': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-01.dcm',
  '1-01.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.184801338381895107635381026857': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-02.dcm',
  '1-02.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.227252232966919521559290115017': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-03.dcm',
  '1-03.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.305763411933569638615761005863': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-04.dcm',
  '1-04.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.892649213333065045357131733969': ('D:/Datasets/Lung-PET-CT-Dxr/m